In [5]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json
import os
from pycocotools.coco import COCO
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (20.0, 20.0)

In [9]:
def get_all_imgs_fldr(img_folder):
    imgs = sorted(os.listdir(img_folder))
    paths = [f'{img_folder}/{f}' for f in imgs]
    assert(int(paths[-1].split('.jpg')[0].split('-')[-1]) == len(paths))
    return paths

In [10]:
def coco2mdNet(video_prefix, annFile, out_fldr= 'mdnet_input', frames_fldr=None, 
               future_frames_fldr='../../../shared',
              results_folder=None):
    """
    # sample input file 
    {"seq_name": videoName_category, 
        "img_list": ["../../shared/shark_frames/20200805_OneSharkSUPFollowsKids/
        20200805_OneSharkSUPFollowsKids-0001.jpg", 
        "init_bbox": [2005, 712, 44, 111], 
        "savefig_dir": "results/20200805_OneSharkSUPFollowsKids_humans", 
        "result_path": "results/20200805_OneSharkSUPFollowsKids_humans/results.json"}
    """
    if not results_folder:
        results_folder = f'results/{video_prefix}'
    
    if not frames_fldr:
        frames_fldr = f'shark_frames/{video_prefix}'
    
    img_paths = get_all_imgs_fldr(frames_fldr)
    img_paths = [f'{future_frames_fldr}/{f}' for f in img_paths]
    # just include the first one 
    coco=COCO(annFile)
    cats = coco.loadCats(coco.getCatIds())
    nms=[cat['name'] for cat in cats]
    os.system(f'mkdir -p {out_fldr}/{video_prefix}')
    extra_annots = {}
    for cat in nms:
        jsonf = {'seq_name': f'{video_prefix}_{cat}', 
                 'savefig_dir': f'{results_folder}_{cat}', 
                 'result_path': f'{results_folder}/results.json', 
                }
        # get coco info 
        catIds = coco.getCatIds(catNms=[cat])
        imgIds = coco.getImgIds(catIds=catIds)
        imgIds = sorted(imgIds)
        if len(imgIds) > 1:
            # load the first one 
            init_img_info = coco.loadImgs(imgIds[0])[0]
            fn = init_img_info['file_name']
            s_ind = init_img_info['id']

            #  get annotations
            annIds = coco.getAnnIds(imgIds=s_ind, catIds=catIds, iscrowd=None)
            anns = coco.loadAnns(annIds)
            if len(anns) > 1:
                print('\t', jsonf['seq_name'], len(anns))
                extra_annots[jsonf['seq_name']] = anns
            jsonf['init_bbox'] = anns[0]['bbox']
            jsonf['img_list'] = img_paths[s_ind:]
            
            with open(f'{out_fldr}/{video_prefix}/{video_prefix}_{cat}.json', 'w') as f:
                json.dump(jsonf, f)

In [11]:
annFldr ='coco_files'
for annFile in os.listdir(annFldr):
    if annFile.endswith('_coco.json'):
#         print(annFile)
        video_pre = annFile.split('_coco.json')[0]
        coco2mdNet(video_pre, f'{annFldr}/{annFile}')

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
	 20210910_SharkSwimmerSurfer_human_0 2
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
	 20210401_ThreeSharksVeryCloseSurfers_human_0 2
	 20210401_ThreeSharksVeryCloseSurfers_human_1 2
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
	 20210615_SharkUnderSurfer_human_0 2
loading annotations into memory...
Done (t=0.00s)
creating index...
index cre